# BlueCruise EDA Mapping: Live Vehicle Testing Investigation (10/30/2024)

## Objective:
- Determine which roads are recording BC data when 'opened up'

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('nbagg')
%matplotlib inline
import folium
from folium.plugins import MarkerCluster
# from google.cloud import bigquery
import os

## GCP Setup:

In [2]:
# # Google Cloud Authentication: 
assert os.system('gcloud auth application-default login --quiet') == 0

In [3]:
# # Insert your project ID here:
PROJECT_ID = "ford-5bba11084fd31e17ec109f0c"

In [4]:
assert os.system(f"gcloud config set core/project {PROJECT_ID}") == 0

In [ ]:
!gcloud config list

In [6]:
from google.cloud import bigquery

### Now lets pull in some initial data to play with

In [20]:
q = """
SELECT 
/* Identifiers */
dpefa_bc_007_trip_d as trip_id,

/*Time*/
dpefa_bc_007_event_local_m as date_time,

/* Driving Characteristics */
dpefa_bc_007_trffc_jam_asst_stat_x as veh_assist_mode,
dpefa_bc_007_hst_veh_long_vlcy_r as veh_long_vel_mps,
dpefa_bc_007_hst_veh_ltrl_vlcy_r as veh_ltrl_vel_mps,
dpefa_bc_007_hst_veh_yaw_rate_r as veh_yaw_rate_radps,
dpefa_bc_007_acc_mps2 as veh_accel_mps2,
dpefa_bc_007_jerk_mps3 as veh_jerk_mps3,

/* Context Clues */
dpefa_bc_007_ehr_mini_form_of_way_x as road_type,

/* GPS */
dpefa_bc_007_gps_long_decm_deg_r_3 as gps_longititude,
dpefa_bc_007_gps_lat_decm_deg_r_3 as gps_latitude,


FROM `prj-dfad-31-usrda-p-31.dlobo1_bluecruise.gdpefa_adas_bc_ada_lm_vw` 
WHERE EXTRACT(date from dpefa_bc_007_event_m) between '2024-10-15' and '2024-10-22'
AND dpefa_bc_007_vin_17_x = '3FMTK3R4XPMA61289'
"""

In [30]:
q = """
SELECT
    DISTINCT dpefa_bc_007_gps_long_decm_deg_r_3 AS gps_longitude,
    dpefa_bc_007_gps_lat_decm_deg_r_3 AS gps_latitude
FROM
    `prj-dfad-31-usrda-p-31.dlobo1_bluecruise.gdpefa_adas_bc_ada_lm_vw` 
WHERE
    EXTRACT(DATE FROM dpefa_bc_007_event_m) BETWEEN '2024-10-15' AND '2024-10-22'
    AND dpefa_bc_007_ehr_mini_form_of_way_x != "CONTROLLED_ACCESS_ROAD"
    AND dpefa_bc_007_gps_long_decm_deg_r_3 BETWEEN -90.4181 AND -82.4135
    AND dpefa_bc_007_gps_lat_decm_deg_r_3 BETWEEN 41.6961 AND 48.3061
"""

In [8]:
q = """
SELECT * 
FROM `ford-5bba11084fd31e17ec109f0c.GATech.bc_gps_data_oct15-oct22`
WHERE gps_longitude BETWEEN -84 AND -82.9
    AND gps_latitude < 43 
    AND gps_latitude > 42.4
"""

#### BigQuery Data Extraction:

In [9]:
# Excecute the query to get 1Hz data for all collision claim vehicles between april and august
client = bigquery.Client()
live_df = client.query(q)
df = live_df.to_dataframe()

In [10]:
df.to_csv(r'C:\Users\dlobo1\Documents\Gatech Research\Jupyter Notebooks\data\bc_livedata_gpsregion_oct15-oct22.csv', index=False)


#### Begin working here: [Don't run the queries and conversions again]

In [2]:
df_bc = pd.read_csv(r'C:\Users\donav\OneDrive - Georgia Institute of Technology\Current Work\data\bc_livedata_gpsregion_oct15-oct22.csv',) # <-- BlueCruise Data

In [ ]:
df_bc.info()

In [ ]:
df_bc.head()

In [ ]:
unique_pairs = df_bc.drop_duplicates(subset=['gps_latitude', 'gps_longitude'])

# Get the number of unique pairs
number_of_unique_pairs = len(unique_pairs)

print(f'The number of unique latitude and longitude pairs is: {number_of_unique_pairs}')

In [6]:
# Randomly sample the data since there are too many points to plot

sampled_df = df_bc.sample(n=100000, random_state=1)

In [ ]:
sampled_df.info()

# Plot GPS coordinates on a map

### Folium interactive map

In [ ]:
# Initialize a folium map centered around the first point
m = folium.Map(location=[sampled_df['gps_latitude'].iloc[0], sampled_df['gps_longitude'].iloc[0]], zoom_start=12)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Add points to the marker cluster
for _, row in sampled_df.iterrows():
    folium.CircleMarker(
        [row['gps_latitude'], row['gps_longitude']],
        radius=2,
        color="blue",
        fill=True,
        fill_opacity=0.6,
    ).add_to(marker_cluster)

# Display the map
m

# Look at a specific area:
- Use the map to determine the region you are interested in (Has to be in the detroit area)
- Input the coordinates of a region of interest
- Use function to find the X-mile bounding box

In [ ]:
## Map to determine coordinates of interest

# Initialize a folium map centered around Detroit
detroit_center = [42.3314, -83.0458]  # Latitude and Longitude for Detroit, MI
m = folium.Map(location=detroit_center, zoom_start=12)

# Add a custom JavaScript function to handle clicks and display coordinates
m.add_child(folium.LatLngPopup())

# Display the map
m

In [25]:
# Enter coordinates of interest: 
lat = 42.571208
long =  -83.472606

In [26]:
def calculate_bounding_box(lat, lon, miles=5):
    # Calculate the degree variations for 5 miles
    # 1 degree of latitude is approximately 69 miles
    lat_variation = miles / 69
    
    # 1 degree of longitude is approximately 69 miles * cos(latitude)
    lon_variation = miles / (69 * np.cos(np.radians(lat)))
    
    # Calculate bounding box coordinates
    northwest = (lat + lat_variation, lon - lon_variation)
    northeast = (lat + lat_variation, lon + lon_variation)
    southwest = (lat - lat_variation, lon - lon_variation)
    southeast = (lat - lat_variation, lon + lon_variation)
    
    return {
        "northwest": northwest,
        "northeast": northeast,
        "southwest": southwest,
        "southeast": southeast
    }

In [27]:
# Calculate the bounding box
bounding_box = calculate_bounding_box(lat, long, miles=5)

# Extract the coordinates from the bounding box
northwest_lat, northwest_lon = bounding_box["northwest"]
southeast_lat, southeast_lon = bounding_box["southeast"]

In [28]:
# Define the bounding box for the region using the calculated values
df_commerce = df_bc[
    (df_bc['gps_longitude'] > northwest_lon) &
    (df_bc['gps_longitude'] < southeast_lon) &
    (df_bc['gps_latitude'] > southeast_lat) &
    (df_bc['gps_latitude'] < northwest_lat)
]

In [29]:
# Randomly sample the data since there are too many points to plot

sampled_df2 = df_commerce.sample(n=10000, random_state=1)

In [ ]:
sampled_df2.info()

In [ ]:
# Initialize a folium map centered around the first point
m = folium.Map(location=[sampled_df2['gps_latitude'].iloc[0], sampled_df2['gps_longitude'].iloc[0]], zoom_start=12)

# Add points to the map from the sampled DataFrame
for _, row in sampled_df2.iterrows():
    folium.CircleMarker(
        [row['gps_latitude'], row['gps_longitude']],
        radius=2,
        color="blue",
        fill=False,
        fill_opacity=0.6,
    ).add_to(m)

# Display the map
m